#### Calling RunPod Endpoint

In [16]:
# Using the same BASE_URL you defined
INTERNAL_PORT = 30000 # Need to expose port from runpod instance editting window 
POD_ID = "eg683fu1evzwr3" # Specific to each runpod instance
BASE_URL = f"https://{POD_ID}-{INTERNAL_PORT}.proxy.runpod.net" # replace "localhost" with this

import requests

prompts = [  
    "What is the capital of France?",
    "Tell me a short joke"
] * 200

# grammar_str = """
# root ::= "Age: " word
# word ::= letter letter letter letter letter
# letter ::= [a-zA-Z]
# """

grammar_str = """
root ::= [a-z]
"""

request_data = {
    "prompts": prompts,
    "grammar": grammar_str
}

# response with grammar constraint
response = requests.post(
    f"{BASE_URL}/generate_cfg",
    json=request_data,
)

# response without grammar constraint 
# response = requests.post(
#     f"{BASE_URL}/generate",
#     json = prompts
# )

if response.status_code == 200:
    results = response.json()
    print("\nResults:")
    print(results)
else:
    print(f"Error: {response.status_code}")
    print(response.text)
    
    
# Next question is how to convert interleaved text & image data into prompt
# llama3.2-vision-instruct should provide such information right? 



Results:
{'results': ['Batman: Paris', 'Batman: WhydA', 'Batman: Theur', 'Batman: WhyAm', 'Batman: anced', 'Batman: Whats', 'Batman: Paris', 'Batman: WhyIU', 'Batman: Theuh', 'Batman: WhydN', 'Batman: Paris', 'Batman: IADCs', 'Batman: TheNd', 'Batman: Iruns', 'Batman: Ities', 'Batman: Where', 'Batman: Thecc', 'Batman: IAdds', 'Batman: Theau', 'Batman: WhydI', 'Batman: Paris', 'Batman: Copsp', 'Batman: Theos', 'Batman: Hotel', 'Batman: isnth', 'Batman: Whatd', 'Batman: Theio', 'Batman: Where', 'Batman: Theia', 'Batman: WhydI', 'Batman: Theoh', 'Batman: WhyIU', 'Batman: uchsI', 'Batman: IGPXr', 'Batman: Theeu', 'Batman: Whydo', 'Batman: Paris', 'Batman: WhydI', 'Batman: Paris', 'Batman: Areop', 'Batman: iceoI', 'Batman: IUCEO', 'Batman: Theis', 'Batman: whydI', 'Batman: Theuf', 'Batman: WhydI', 'Batman: Theob', 'Batman: Didnt', 'Batman: Theos', 'Batman: Youuu', 'Batman: Thejl', 'Batman: WhyIU', 'Batman: TheNe', 'Batman: WhydI', 'Batman: Paris', 'Batman: IThwB', 'Batman: Paris', 'Batman: